In [1]:
""" 라이브러리 불러오기 """
import requests
import numpy as np
import warnings
import re
import sys
import json
import time
import datetime 

import folium
from folium.plugins import MiniMap

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from bs4 import BeautifulSoup
from collections import OrderedDict

warnings.filterwarnings(action='ignore')

In [2]:
""" 함수 정의 """

""" 법정동을 행정동으로 변환하는 함수 """
def geo(i):
    # 법정동 -> 행정동
    i['행정동'] = i['행정동'].str.replace('산울동','도담동')
    i['행정동'] = i['행정동'].str.replace('해밀동','도담동')
    i['행정동'] = i['행정동'].str.replace('집현동','소담동')
    i['행정동'] = i['행정동'].str.replace('어진동','도담동')
    i['행정동'] = i['행정동'].str.replace('나성동','새롬동')
    i['행정동'] = i['행정동'].str.replace('가람동','한솔동')
    i['행정동'] = i['행정동'].str.replace('반곡동','소담동')
    i['행정동'] = i['행정동'].str.replace('합강동','소담동')
    
""" 주소에서 시, 행정동 추출하는 함수 """
def create_dong(df):
    df['시'] = df['address_name'].str.split(" ").str[0]
    df['행정동'] = df['address_name'].str.split(" ").str[1]


""" 특정 행정동(도담동, 새롬동, 조치원읍)을 추출하는 함수 """
def select_dong(df):
    conditions = (df["행정동"] == "도담동")|(df["행정동"] == "새롬동")|(df["행정동"] == "조치원읍")
    return conditions

""" 주소에서 위도경도 추출하는 함수 """
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + "57a426ba3fb221a28bbb05da0282184d"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    try:
        match_first = result['documents'][0]['address']
        return float(match_first['x']), float(match_first['y'])
    except IndexError:
        return '없음', '없음'

# 데이터리스트

① 육아시설 : 어린이집 + 유치원
② 학교 : 초등학교 + 중학교 + 고등학교 + 대학교
③ 음식점
④ 카페
⑤ 주거
⑥ 편의점
⑦ 주차장


## ① 육아시설

In [3]:
""" 데이터 불러오기 """
minwon = pd.read_csv("minwon.csv", header = 10)
minwon.head()

,유형,기관명,인가일,통학차량,행정동,주소,평가인증,정원,아동현원,Unnamed: 9,Unnamed: 10
0,민간,i키움어린이집,2017-09-04,미운영,보람동,세종특별자치시 시청대로 210 호려울마을 10단지 관리동,N,63,33,044-867-6054,NaN
1,공립,가득유치원,2017-02-15,미운영,새롬동,세종특별자치시 세종로 933,Y,168,162,044-999-6800,NaN
2,국공립,가락마을어린이집,2020-10-07,미운영,고운동,"세종특별자치시 마음로 49 관리동 (고운동, 가락마을2단지)",N,34,30,044-862-0057,NaN
3,공립,가락유치원,2015-09-01,미운영,고운동,세종특별자치시 마음안1로 22,Y,187,176,044-903-1300,NaN
4,민간,가락키즈어린이집,2015-10-30,미운영,고운동,세종특별자치시 마음안로 47 가락마을3단지 관리동,N,47,45,044-999-0333,NaN


In [4]:
""" 특정 행정동 추출 """
lonlat_minwon = minwon[select_dong(minwon)].sort_values(by = ['행정동']).reset_index(drop = True)
lonlat_minwon.head()

,유형,기관명,인가일,통학차량,행정동,주소,평가인증,정원,아동현원,Unnamed: 9,Unnamed: 10
0,국공립,힐스누리어린이집,2020-12-01,미운영,도담동,세종특별자치시 보듬2로 43 도램마을15단지 관리동,N,46,45,044-868-5305,NaN
1,직장,윤빛어린이집,2013-12-09,미운영,도담동,세종특별자치시 한누리대로 402 산업통상자원부동,Y,195,149,044-715-7005,NaN
2,가정,세종도담도담어린이집,2014-04-28,미운영,도담동,세종특별자치시 다솜1로 140 극동스타클래스 104-104,Y,20,17,044-867-5525,NaN
3,국공립,마스터힐스어린이집,2020-11-01,미운영,도담동,"세종특별자치시 해밀3로 32 117동 (해밀동, 해밀마을1단지)",N,45,43,044-865-8300,NaN
4,가정,우리들의어린이집,2017-04-04,미운영,도담동,"세종특별자치시 보듬4로 9 1101동203호(도담동, 도램마을11단지)",Y,18,16,044-866-2367,NaN


In [5]:
""" 위도경도 변수 생성 """
lst = lonlat_minwon['주소'].index
lonlat_minwon['경도'] = None
lonlat_minwon['위도'] = None

for i in lst:
    lonlat = addr_to_lat_lon(lonlat_minwon['주소'][i])
    lonlat_minwon['경도'][i] = lonlat[0]
    lonlat_minwon['위도'][i] = lonlat[1]

In [6]:
""" 위도경도 변환되지 않은 경우 체크 """
lonlat_minwon[(lonlat_minwon['경도'] == '없음' )|(lonlat_minwon['위도'] == '없음')]

,유형,기관명,인가일,통학차량,행정동,주소,평가인증,정원,아동현원,Unnamed: 9,Unnamed: 10,경도,위도
28,가정,꼬마대통령어린이집,2015-01-09,미운영,도담동,세종특별자치시 보듬4로 20 1003-102 도램마을10단지,Y,20,20,044-867-9630,NaN,없음,없음
87,사립,성모유치원,1979-03-05,운영(신고),조치원읍,세종특별자치시 조치원읍 시내6길 30,Y,62,43,044-863-5322,NaN,없음,없음
100,민간,세종영재어린이집,2005-06-01,운영(신고),조치원읍,세종특별자치시 조치원읍 평안로 36-26,Y,99,99,044-864-6798,NaN,없음,없음


In [7]:
""" 위도경도 수동변환 """
# 이용 사이트 : http://www.dawuljuso.com/

# 꼬마대통령어린이집  # X :127.262342, Y :36.5125239
lonlat_minwon.iloc[28,11] = '127.262342'
lonlat_minwon.iloc[28,12] = '36.5125239'

# 성모유치원  # X :127.300203, Y :36.5980885 (시내 → 새내)
lonlat_minwon.iloc[87,11] = '27.300203'
lonlat_minwon.iloc[87,12] = '36.5980885'

# 세종영재어린이집  # X :127.298313, Y :36.5890188
lonlat_minwon.iloc[100,11] = '127.298313'
lonlat_minwon.iloc[100,12] = '36.5890188'

In [8]:
""" 필요변수 추출 """
lonlat_minwon = lonlat_minwon.loc[:,['기관명','주소','경도','위도']].sort_values(by = ['기관명']).reset_index(drop = True)
lonlat_minwon.columns = ['육아시설','주소','경도','위도']

lonlat_minwon.head() # (109, 4)

,육아시설,주소,경도,위도
0,가득유치원,세종특별자치시 세종로 933,127.243521,36.487056
1,가온유치원,세종특별자치시 보람로 71,127.262371,36.514655
2,개구쟁이어린이집,세종특별자치시 조치원읍 산막길17 우방유쉘 103-101,127.295746,36.593589
3,근로복지공단세종어린이집,세종특별자치시 조치원읍 새내2길 41,127.301066,36.597231
4,꼬마대통령어린이집,세종특별자치시 보듬4로 20 1003-102 도램마을10단지,127.262342,36.5125239


In [9]:
""" 데이터 내보내기 """
lonlat_minwon.to_csv("lonlat_minwon.csv")

## ② 학교

In [10]:
""" 데이터 불러오기 """
ele = pd.read_csv("세종특별자치시교육청+행정지원과_210301_초등학교.csv", header = 3)  # (51, 7)
mid = pd.read_csv("세종시교육청+중학교+현황(2021.07.01.)+.csv", header = 3)  # (26, 7)
high = pd.read_csv("세종특별자치시교육청+행정지원과_210301_고등학교.csv", header = 3)  # (21, 7)
coll = pd.read_csv("세종시_대학교.csv")  # (4, 7)

In [11]:
""" 데이터 합치기 """
school = pd.concat([ele,mid,high,coll]).reset_index(drop = True)
school.head()  # (102, 7)

,순번,구분,기관명,전화번호,팩스번호,주소,홈페이지
0,1,공립,조치원대동초등학교,861-4603,867-3423,세종특별자치시 조치원읍 대동학교길 11,http://jdd.sjedues.kr/
1,2,공립,조치원명동초등학교,866-5786,866-9990,세종특별자치시 조치원읍 조치원10길 35,http://jmd.sjedues.kr
2,3,공립,조치원교동초등학교,863-6472,863-6473,세종특별자치시 조치원읍 새내18길 21,http://jk.sjedues.kr
3,4,공립,조치원신봉초등학교,902-1400,866-9991,세종특별자치시 조치원읍 서북부로 10,http://jcwsb.sjedues.kr
4,5,공립,연남초등학교,863-4500,863-5717,세종특별자치시 연기면 연기길 2,http://yeonnam.sjedues.kr


In [12]:
lonlat_school = school.copy()

""" 위도경도 변수 생성 """
lst = lonlat_school['주소'].index
lonlat_school['경도'] = None
lonlat_school['위도'] = None

for i in lst:
    lonlat = addr_to_lat_lon(lonlat_school['주소'][i])
    lonlat_school['경도'][i] = lonlat[0]
    lonlat_school['위도'][i] = lonlat[1]

In [13]:
""" 위도경도 변환되지 않은 경우 체크 """
lonlat_school[(lonlat_school['경도'] == '없음' )|(lonlat_school['위도'] == '없음')]

,순번,구분,기관명,전화번호,팩스번호,주소,홈페이지,경도,위도


In [14]:
""" 필요변수 추출 """
lonlat_school = lonlat_school.loc[:,['기관명','주소','경도','위도']].sort_values(by = ['기관명']).reset_index(drop = True)
lonlat_school.columns = ['학교','주소','경도','위도']

lonlat_school.head()  # (102, 4)

,학교,주소,경도,위도
0,가득초등학교,세종특별자치시 세종로 941,127.243231,36.487716
1,가락초등학교,세종특별자치시 마음로 208,127.23084,36.515685
2,감성초등학교,세종특별자치시 금남면 감성길 6,127.289365,36.443659
3,고려대학교 세종캠퍼스,세종특별자치시 조치원읍 세종로 2511,127.291003,36.611695
4,고운고등학교,세종특별자치시 마음안로 93,127.233078,36.510229


In [15]:
""" 데이터 내보내기 """
lonlat_school.to_csv("lonlat_school.csv")

## ③ 음식점

In [16]:
""" 데이터 불러오기 """
eat = pd.read_csv('eat.csv')
eat.head()  # (6007, 7)

,Unnamed: 0,ID,Category_code,name,X,Y,address_name
0,0,8935802,FD6,뚱땡이감자탕,127.135131,36.449595,충남 공주시 옥룡동 76
1,1,10636281,FD6,오룡반점,127.134796,36.449206,충남 공주시 옥룡동 98-4
2,2,11230780,FD6,투다리 옥룡점,127.134997,36.449848,충남 공주시 옥룡동 75-4
3,3,18819294,FD6,맥시칸치킨 옥룡점,127.134796,36.449217,충남 공주시 옥룡동 98-4
4,4,8678530,FD6,간이역,127.135234,36.449429,충남 공주시 옥룡동 76-4


In [17]:
""" 시, 행정동 변수 생성 """
create_dong(eat)

""" 세종시 데이터 추출 """
sejong_eat = eat[eat['시'] == '세종특별자치시'].reset_index(drop=True)

""" 법정동을 행정동으로 변환 """
geo(sejong_eat)

""" 특정동 추출 """
lonlat_eat = sejong_eat[select_dong(sejong_eat)]

""" 필요변수 추출 """
lonlat_eat = lonlat_eat.loc[:,['name','address_name','X','Y']]
lonlat_eat.columns = ['음식점','주소','경도','위도']

lonlat_eat.head()  # (1788, 4)

,음식점,주소,경도,위도
481,이동국상황족발,세종특별자치시 새롬동 329,127.247278,36.486831
482,만년닭강정 새롬점,세종특별자치시 새롬동 305-1,127.249988,36.487348
483,청년피자 새롬점,세종특별자치시 새롬동 산 40,127.247151,36.488812
484,바른치킨 세종새롬점,세종특별자치시 새롬동 554,127.247489,36.486891
485,지코바 새롬점,세종특별자치시 새롬동 554,127.246954,36.489386


In [18]:
""" 데이터 내보내기 """
lonlat_eat.to_csv('lonlat_eat.csv')

## ④ 카페

In [19]:
""" 데이터 불러오기 """
cafe = pd.read_csv('cafe.csv')
cafe.head()  # (1216, 7)

,Unnamed: 0,ID,Category_code,name,X,Y,address_name
0,0,725636525,CE7,골목카페동주,127.133939,36.448868,충남 공주시 옥룡동 163-2
1,1,826713592,CE7,카페에덴,127.135837,36.447446,충남 공주시 옥룡동 101
2,2,1504836887,CE7,카페여녹,127.135875,36.456392,충남 공주시 옥룡동 51-1
3,3,468505097,CE7,봄날,127.133799,36.452704,충남 공주시 옥룡동 180
4,4,2027180273,CE7,카페로뎀나무,127.130592,36.455488,충남 공주시 옥룡동 284-18


In [20]:
""" 시, 행정동 변수 생성 """
create_dong(cafe)

""" 세종시 데이터 추출 """
sejong_cafe = cafe[cafe['시'] == '세종특별자치시'].reset_index(drop=True)

""" 법정동을 행정동으로 변환 """
geo(sejong_cafe)

""" 특정동 추출 """
lonlat_cafe = sejong_cafe[select_dong(sejong_cafe)]

""" 필요변수 추출 """
lonlat_cafe = lonlat_cafe.loc[:,['name','address_name','X','Y']]
lonlat_cafe.columns = ['카페','주소','경도','위도']

lonlat_cafe.head()  # (365, 4)

,카페,주소,경도,위도
109,카페온,세종특별자치시 새롬동 329,127.247235,36.486275
110,커피홀 세종새롬점,세종특별자치시 새롬동 328,127.247745,36.487196
204,공차 세종새롬점,세종특별자치시 새롬동 592,127.251794,36.484179
205,두헤브마카롱,세종특별자치시 새롬동 341-105,127.251835,36.484050
206,빽다방 세종새롬중앙점,세종특별자치시 새롬동 산 75,127.250784,36.485007


In [21]:
""" 데이터 내보내기 """
lonlat_cafe.to_csv('lonlat_cafe.csv')

## ⑤ 주거

In [33]:
""" 데이터 불러오기 """
apt = pd.read_csv('apt.csv')  # (800, 7)
office = pd.read_csv('office.csv')  # (559, 7)
villa = pd.read_csv('villa.csv')  # (612, 7)
house = pd.read_csv('house.csv')  # (643, 7)

In [34]:
""" 데이터 합치기 """
house = pd.concat([apt,office,villa,house]).reset_index(drop = True)
house.head()  # (2614, 7)

,Unnamed: 0,ID,name,X,Y,road_address,address_name
0,0,11361049,덕성그린타운아파트,127.136391,36.447996,충남 공주시 우금티로 704,충남 공주시 옥룡동 93
1,1,11149309,주공아파트,127.134275,36.447404,충남 공주시 우금티로 709-11,충남 공주시 옥룡동 103
2,2,8017276,옥룡주공1단지아파트,127.134080,36.448179,충남 공주시 우금티로 709-10,충남 공주시 옥룡동 123
3,3,1679412760,옥룡주공아파트 2단지관리사무실,127.135244,36.447612,충남 공주시 우금티로 709-11,충남 공주시 옥룡동 103
4,4,10618366,옥룡 주공1단지아파트 노인회,127.134013,36.448096,충남 공주시 우금티로 709-14,충남 공주시 옥룡동 123


In [35]:
""" 시, 행정동 변수 생성 """
create_dong(house)

""" 세종시 데이터 추출 """
sejong_house = house[house['시'] == '세종특별자치시'].reset_index(drop=True)

""" 법정동을 행정동으로 변환 """
geo(sejong_house)

""" 특정동 추출 """
lonlat_house = sejong_house[select_dong(sejong_house)]

""" 필요변수 추출 """
lonlat_house = lonlat_house.loc[:,['name','address_name','X','Y']]
lonlat_house.columns = ['주거','주소','경도','위도']

lonlat_house.head()  # (644, 4)

,주거,주소,경도,위도
53,새뜸마을1단지메이저시티아파트,세종특별자치시 새롬동 산 78,127.247569,36.481004
54,새뜸마을2단지메이저시티아파트,세종특별자치시 새롬동 산 78,127.247712,36.483572
55,새뜸마을6단지메이저시티아파트 601동,세종특별자치시 새롬동 593,127.249766,36.483659
56,새뜸마을3단지아파트 307동,세종특별자치시 새롬동 329,127.246502,36.484632
58,새뜸마을4단지캐슬앤파밀리에아파트 417동,세종특별자치시 새롬동 373,127.244535,36.489016


In [38]:
""" 데이터 중복 제거 """
lonlat_house = lonlat_house.drop_duplicates(['주거'], keep = 'first')

""" 데이터 정제 """
ex_index = lonlat_house.query('주거.str.contains("예정|오피스텔|전기차충전소|상가|경비실|관리사무소|관리소|지하주차장|경로당|정문|후문|쏘카존|도서관|조합|사무처|본부|협회|기금|금융|나눔터|건설|산업|복지관")', engine='python').index
lonlat_house = lonlat_house.drop(index = ex_index).reset_index(drop=True)

lonlat_house.head()  # (260, 4)

,주거,주소,경도,위도
0,새뜸마을1단지메이저시티아파트,세종특별자치시 새롬동 산 78,127.247569,36.481004
1,새뜸마을2단지메이저시티아파트,세종특별자치시 새롬동 산 78,127.247712,36.483572
2,새뜸마을6단지메이저시티아파트 601동,세종특별자치시 새롬동 593,127.249766,36.483659
3,새뜸마을3단지아파트 307동,세종특별자치시 새롬동 329,127.246502,36.484632
4,새뜸마을4단지캐슬앤파밀리에아파트 417동,세종특별자치시 새롬동 373,127.244535,36.489016


In [25]:
""" 데이터 내보내기 """
lonlat_house.to_csv('lonlat_house.csv')

## ⑥ 편의점

In [26]:
""" 데이터 불러오기 """
con = pd.read_csv("convenience.csv", index_col = 0)
con.head()  # (494, 6)

,ID,name,X,Y,road_address,address_name
0,17056961,GS25 옥룡주공점,127.134792,36.449295,충남 공주시 우금티로 719,충남 공주시 옥룡동 98-4
1,19201407,CU 공주미도점,127.135345,36.449266,충남 공주시 우금티로 720,충남 공주시 옥룡동 76-5
2,26937138,GS25 공주옥룡점,127.135124,36.455964,충남 공주시 무령로 318,충남 공주시 옥룡동 50-3
3,1966822207,CU 공주옥룡로드점,127.131991,36.455604,충남 공주시 무령로 290,충남 공주시 옥룡동 284-6
4,8664932,CU 공주옥룡점,127.133444,36.454268,충남 공주시 우금티로 777,충남 공주시 옥룡동 331-31


In [27]:
""" 시, 행정동 변수 생성 """
create_dong(con)

""" 세종시 데이터 추출 """
sejong_con = con[con['시'] == '세종특별자치시'].reset_index(drop=True)

""" 법정동을 행정동으로 변환 """
geo(sejong_con)

""" 특정동 추출 """
lonlat_con = sejong_con[select_dong(sejong_con)]

""" 필요변수 추출 """
lonlat_con = lonlat_con.loc[:,['name','address_name','X','Y']]
lonlat_con.columns = ['편의점','주소','경도','위도']

lonlat_con.head()  # (137, 4)

,편의점,주소,경도,위도
45,이마트에브리데이 세종메이저점,세종특별자치시 새롬동 산 78,127.246778,36.479937
46,GS25 세종메이저점,세종특별자치시 새롬동 341-144,127.249307,36.483756
47,GS25 세종캐슬점,세종특별자치시 새롬동 산 40,127.247137,36.488699
48,이마트24 세종캐슬점,세종특별자치시 새롬동 328,127.247705,36.487075
84,GS25 세종새롬점,세종특별자치시 새롬동 579,127.250817,36.486875


In [28]:
""" 데이터 내보내기 """
lonlat_con.to_csv("lonlat_convenience.csv")

## ⑦ 주차장

In [29]:
""" 데이터 불러오기 """
parking = pd.read_csv("parking.csv", index_col = 0)
parking.head()  # (457, 6)

,ID,name,X,Y,road_address,address_name
0,20595952,지하주차장,127.135962,36.447806,충남 공주시 우금티로 704,충남 공주시 옥룡동 93
1,17570522,공주대학교옥룡캠퍼스 주차장2,127.133008,36.449871,NaN,충남 공주시 옥룡동 191-3
2,17574443,공주대학교 옥룡캠퍼스 주차장1,127.133156,36.450689,NaN,충남 공주시 옥룡동 187-4
3,18984049,신관금강공원 주차장,127.132423,36.466695,NaN,충남 공주시 신관동 553
4,17557065,공주종합버스터미널 주차장,127.134776,36.468411,충남 공주시 신관로 74,충남 공주시 신관동 609


In [30]:
""" 시, 행정동 변수 생성 """
create_dong(parking)

""" 세종시 데이터 추출 """
sejong_parking = parking[parking['시'] == '세종특별자치시'].reset_index(drop=True)

""" 법정동을 행정동으로 변환 """
geo(sejong_parking)

""" 특정동 추출 """
lonlat_parking = sejong_parking[select_dong(sejong_parking)]

""" 필요변수 추출 """
lonlat_parking = lonlat_parking.loc[:,['name','address_name','X','Y']]
lonlat_parking.columns = ['주차장','주소','경도','위도']

lonlat_parking.head()  # (169, 4)

,주차장,주소,경도,위도
25,은하수공원 제2주차장,세종특별자치시 산울동 680,127.239598,36.535603
48,은하수공원 제1주차장,세종특별자치시 산울동 680,127.245144,36.532279
49,은하수공원 화장장 지하주차장,세종특별자치시 산울동 680,127.245556,36.533987
50,은하수공원 봉안당 주차장,세종특별자치시 산울동 680,127.245257,36.534889
51,은하수공원 버스 리무진 전용주차장,세종특별자치시 산울동 680,127.245434,36.534528


In [31]:
""" 데이터 내보내기 """
lonlat_parking.to_csv("lonlat_parking.csv")